In [1]:
import os
import json
import time
import pandas as pd
from IPython.display import display

from lkae.retrieval.retrieve import get_retriever, retrieve_evidence, AuredDataset
from lkae.utils.data_loading import pkl_dir, load_pkl, root_dir

import pyterrier as pt
import pyterrier.io as ptio
import pyterrier.pipelines as ptpipelines
from ir_measures import R, MAP    

if not pt.started():
    pt.init()

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8



In [2]:
datasets = {}

# walk through the pkl directory and load all the datasets in one of its subdirectories
# load each dataset with its subdirectory name and filename as the key
# skip non-pkl files
for subdir in os.listdir(pkl_dir):
    if not os.path.isdir(os.path.join(pkl_dir, subdir)):
        continue            
    datasets[subdir] = {}
    for filename in os.listdir(os.path.join(pkl_dir, subdir)):
        if not filename.endswith('.pkl'):
            continue
        key = os.path.join(subdir, filename)
        datasets[subdir][filename.split('.')[0]] = load_pkl(os.path.join(pkl_dir, key))

# possilbe splits: train, dev, train_dev_combined
# (test, all_combined don't have "labels")
split = 'train_dev_combined'

dataset_split = f'English_{split}'
qrel_filename = f'{dataset_split}_qrels.txt'

dataset_variations_dict = datasets[dataset_split]
print(dataset_variations_dict.keys())

# ground truth RQ1
golden = ptio.read_qrels(os.path.join(root_dir, 'data', qrel_filename))

# select a set of variations of the dataset
# these selected variations are selected for these reasons:
# - pre-nonam-nobio     ("raw" data, but preprocessed)
# - pre-nam-bio         (we would expect lexical retrieval to be best here)
# - nopre-nonam-nobio   ("raw" data)
# - nopre-nam-bio       (we would expect semantic retrieval to be best here, most information contained here)
selected_variations = ["pre-nonam-nobio", "pre-nam-bio", "nopre-nonam-nobio", "nopre-nam-bio"]


dict_keys(['nopre-nam-bio', 'nopre-nam-nobio', 'nopre-nonam-bio', 'nopre-nonam-nobio', 'pre-nam-bio', 'pre-nam-nobio', 'pre-nonam-bio', 'pre-nonam-nobio'])


In [3]:
# load each config and construct its retriever

retrievers = {}

with open('config.json', 'r') as file:
    configs = json.load(file)

    for config in configs['configs']:
        retriever_label = get_retriever(**config)
        retrievers[config['retriever_method']] = retriever_label

retrievers

Initializing HFSentenceTransformersRetriever with model: sentence-transformers/multi-qa-distilbert-cos-v1
Waiting for model to warm up (for 20.0 seconds)
Initializing CrossEncoderRetriever with model: cross-encoder/stsb-roberta-large


{'bm25': <lkae.retrieval.methods.bm25.BM25Retriever at 0x191d6221510>,
 'openai': <lkae.retrieval.methods.openai_embeddings.OpenAIRetriever at 0x191d6223a90>,
 'sent-transformers-hf': <lkae.retrieval.methods.sent_transformers_hf.HFSentenceTransformersRetriever at 0x1921874b370>,
 'sent-transformers-local': <lkae.retrieval.methods.sent_transformers.SBERTRetriever at 0x19218748d90>,
 'crossencoder': <lkae.retrieval.methods.sent_transformers.CrossEncoderRetriever at 0x1922481e410>,
 'tfidf': <lkae.retrieval.methods.tfidf.TFIDFRetriever at 0x19225debf40>}

In [4]:
# then for every variation of the dataset in ds, run the experiment with each retriever and save the results

out_dir = 'results'
data = []

for selected_variation in selected_variations:
    dataset: AuredDataset = dataset_variations_dict[selected_variation]
    for retriever_label in retrievers:
        start = time.time()

        retrieved_data = retrieve_evidence(dataset[:], retrievers[retriever_label])

        pred = pd.DataFrame([[*d, retriever_label] for d in retrieved_data], columns=['qid', 'docno', 'rank', 'score', 'name']) 

        eval = ptpipelines.Evaluate(pred, golden, metrics = [R@5,MAP], perquery=False)
        r5, meanap = [v for v in eval.values()]

        score = r5

        wall_time = time.time() - start

        print(f'result for retrieval run - R@5: {r5:.4f} MAP: {meanap:.4f} with config\tretriever: {retriever_label};\tds: {selected_variation}, took {wall_time:.2f} seconds')
        
        data.append({
            'R5': r5,
            'MAP': meanap,
            'Retrieval_Method': retriever_label, 
            'DS_Settings': selected_variation,
            'Time (s)': wall_time,
        })

# Convert the list of dictionaries to a DataFrame
df_retrieval = pd.DataFrame(data)

df_retrieval.to_csv(f'{out_dir}/df_retrieval.csv')
print(f'saved df to {out_dir}/df_retrieval.csv')

# Display the DataFrame
display(df_retrieval.sort_values(by='R5', ascending=False))

result for retrieval run - R@5: 0.6345 MAP: 0.5679 with config	retriever: bm25;	ds: pre-nonam-nobio, took 2.07 seconds
result for retrieval run - R@5: 0.6186 MAP: 0.5719 with config	retriever: openai;	ds: pre-nonam-nobio, took 143.42 seconds
result for retrieval run - R@5: 0.5972 MAP: 0.5525 with config	retriever: sent-transformers-hf;	ds: pre-nonam-nobio, took 303.73 seconds


c:\Users\luisk\miniconda3\envs\thesis\lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


result for retrieval run - R@5: 0.5968 MAP: 0.5550 with config	retriever: sent-transformers-local;	ds: pre-nonam-nobio, took 20.21 seconds
result for retrieval run - R@5: 0.6433 MAP: 0.5691 with config	retriever: crossencoder;	ds: pre-nonam-nobio, took 681.92 seconds
result for retrieval run - R@5: 0.6022 MAP: 0.5045 with config	retriever: tfidf;	ds: pre-nonam-nobio, took 0.97 seconds
result for retrieval run - R@5: 0.6457 MAP: 0.5809 with config	retriever: bm25;	ds: pre-nam-bio, took 2.88 seconds
result for retrieval run - R@5: 0.6339 MAP: 0.5756 with config	retriever: openai;	ds: pre-nam-bio, took 183.76 seconds
Waiting for model to warm up (for 20.0 seconds)
result for retrieval run - R@5: 0.5542 MAP: 0.5076 with config	retriever: sent-transformers-hf;	ds: pre-nam-bio, took 481.84 seconds
result for retrieval run - R@5: 0.5681 MAP: 0.5204 with config	retriever: sent-transformers-local;	ds: pre-nam-bio, took 25.70 seconds
result for retrieval run - R@5: 0.6025 MAP: 0.5325 with config

,R5,MAP,Retrieval_Method,DS_Settings,Time (s)
6,0.645675,0.580895,bm25,pre-nam-bio,2.881830
4,0.643261,0.569146,crossencoder,pre-nonam-nobio,681.923324
18,0.637772,0.565110,bm25,nopre-nam-bio,3.133084
0,0.634475,0.567864,bm25,pre-nonam-nobio,2.066375
7,0.633950,0.575602,openai,pre-nam-bio,183.755365
13,0.631840,0.588819,openai,nopre-nonam-nobio,182.610973
19,0.628753,0.568285,openai,nopre-nam-bio,165.777771
12,0.620712,0.558793,bm25,nopre-nonam-nobio,2.256596
23,0.618952,0.531911,tfidf,nopre-nam-bio,1.360650
1,0.618600,0.571916,openai,pre-nonam-nobio,143.422597
